     |████████████████████████████████| 2.9 MB 3.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, regexp_extract, split
import psycopg2

# 1. Créer une session Spark
print("Création de la session Spark...")
spark = SparkSession.builder \
    .appName("PostgresToSpark") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.3.9.jar") \
    .getOrCreate()

# 2. Configurer les paramètres de connexion
print("Configuration des paramètres de connexion JDBC pour PostgreSQL...")
jdbc_url = "jdbc:postgresql://postgres:5432/DB_Mastodon"
properties = {
    "user": "fadi",       
    "password": "fadi",   
    "driver": "org.postgresql.Driver"  
}

# 3. Lire les données depuis PostgreSQL
print("Lecture des données depuis la table 'Mostodon_BRONZ_bis' dans PostgreSQL...")
df = spark.read.jdbc(url=jdbc_url, table="Mostodon_BRONZ_bis", properties=properties)
print(f"Nombre de lignes lues : {df.count()}")

# 4. Nettoyer la colonne 'content' des balises HTML
print("Suppression des balises HTML dans la colonne 'content'...")
df_cleaned = df.withColumn("content", regexp_replace(col("content"), "<[^>]+>", ""))

# 5. Extraire les hashtags (mots précédés de #) et ajouter une nouvelle colonne 'tags'
print("Extraction des hashtags depuis la colonne 'content'...")
df_with_tags = df_cleaned.withColumn("tags", regexp_extract(col("content"), r'(#\w+)', 0))

# Si un contenu peut avoir plusieurs hashtags, on peut les séparer en utilisant 'split'
df_with_tags = df_with_tags.withColumn("tags", split(col("tags"), " "))

# 6. Filtrer les lignes avec certaines conditions
print("Filtrage des données en fonction des colonnes 'favourites_count', 'reblogs_count', 'replies_count'...")
df_max_values = df_with_tags.filter(
    (col("favourites_count") > 1) & 
    (col("reblogs_count") > 1) & 
    (col("replies_count") > 1)
)
print(f"Nombre de lignes après filtrage : {df_max_values.count()}")

# 7. Afficher les résultats filtrés
df_max_values.show()

# 8. Connexion à PostgreSQL via psycopg2
print("Connexion à PostgreSQL avec psycopg2...")
conn = psycopg2.connect(
    host="postgres",  
    database="DB_Mastodon",
    user="fadi",
    password="fadi"
)
cursor = conn.cursor()

# 9. Création de la table 'Mostodon_SILVER_bis' avec une colonne pour les 'tags'
print("Création de la table 'Mostodon_SILVER_bis' avec une colonne 'tags' si elle n'existe pas déjà...")
create_table_query = '''
CREATE TABLE IF NOT EXISTS Mostodon_SILVER_bis (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL,
    tags TEXT  -- Colonne pour stocker les hashtags
);
'''
cursor.execute(create_table_query)
conn.commit()

# 10. Lire les IDs existants dans la table Mostodon_SILVER_bis
existing_ids = spark.read.jdbc(url=jdbc_url, table="Mostodon_SILVER_bis", properties=properties) \
    .select("id") \
    .rdd.flatMap(lambda x: x).collect()

# 11. Filtrer le DataFrame pour exclure les enregistrements existants
df_filtered = df_max_values.filter(~col("id").isin(existing_ids))

# 12. Insertion des données filtrées dans la table Mostodon_SILVER_bis
print("Insertion des données nettoyées avec les tags dans la table 'Mostodon_SILVER_bis' via Spark JDBC...")

# Convertir la colonne tags (array) en texte avant d'écrire dans la base
df_filtered = df_filtered.withColumn("tags", col("tags").cast("string"))

# Insertion des données
df_filtered.write.jdbc(url=jdbc_url, table="Mostodon_SILVER_bis", mode="append", properties=properties)
print("Insertion terminée.")

# 13. Compter le nombre total d'entrées dans la table 'Mostodon_SILVER_bis'
print("Calcul du nombre total d'entrées dans la table 'Mostodon_SILVER_bis'...")
cursor.execute("SELECT COUNT(*) FROM Mostodon_SILVER_bis;")
count = cursor.fetchone()[0]
print(f"Nombre total d'entrées dans la table 'Mostodon_SILVER_bis' : {count}")

# 14. Fermer le curseur et la connexion PostgreSQL
cursor.close()
conn.close()
print("Connexion à PostgreSQL fermée.")

# 15. Fermer la session Spark
print("Fermeture de la session Spark...")
spark.stop()
print("Session Spark fermée.")


Création de la session Spark...
Configuration des paramètres de connexion JDBC pour PostgreSQL...
Lecture des données depuis la table 'Mostodon_BRONZ_bis' dans PostgreSQL...
Nombre de lignes lues : 94855
Suppression des balises HTML dans la colonne 'content'...
Extraction des hashtags depuis la colonne 'content'...
Filtrage des données en fonction des colonnes 'favourites_count', 'reblogs_count', 'replies_count'...
Nombre de lignes après filtrage : 92145
+------------------+------------------+--------------------+--------------------+----------------+-------------+-------------+------------+
|                id|          username|        display_name|             content|favourites_count|reblogs_count|replies_count|        tags|
+------------------+------------------+--------------------+--------------------+----------------+-------------+-------------+------------+
|113284756705098796|       CACGraphics|        CAC Graphics|“Keeneland Backst...|              46|           65|         